In [3]:
!which python3
!which pip
!pip show sqlite3

/home/tonydevs/learn/sql/sql_proj_1/.venv_sql_proj_1/bin/python3


/home/tonydevs/learn/sql/sql_proj_1/.venv_sql_proj_1/bin/pip


References: 

- ref 1: https://docs.python.org/3.10/library/sqlite3.html
- ref 2: https://peps.python.org/pep-0249/
- ref 3: https://wiki.python.org/moin/DatabaseProgramming
- ref 4: https://wiki.python.org/moin/DatabaseInterfaces


### 1. **Connection** Object 
`con = sqlite3.connect("tutorial.db")`

#### 1.1 **Connection Object** (`con`) 
The **Connection Object** (`con`) is created:
- The returned object `con` points to **Tutorial Database** (`tutorial.db`)
- It represents the *connection* to the on-disk database.

#### 1.2 **Tutorial Database** (`tutorial.db`)
The **Tutorial Database** (`tutorial.db`) is ***created*** if it didn't exist.

In [ ]:
import sqlite3
con = sqlite3.connect("tutorial.db")
con # <sqlite3.Connection at ...>

### 2. **Database Cursor**
`cur = con.cursor()`

#### 2.1 The **Database Cursor** (`con.cursor()`)
The `cur` object is ***required*** to: 
- *execute* `SQL` statements and 
- *fetch* results

In [18]:
cur = con.cursor()
cur # <sqlite3.Cursor at ...>

### 3. **Execute** SQL COMMANDS
Use **database cursor** (`cur`) to **execute** SQL commands:
- `cur.execute(...)`


#### 3.1. Creating a Table (`CREATE TABLE ...`):
***Optional*** data-types (flexible typing feature):
- `cur.execute("CREATE TABLE movie(title, year, score)")`

#### 3.2 Verify With `sqlite_master` Table 
`res = cur.execute("SELECT name FROM sqlite_master")` then `res.fetchone()`

`sqlite_master` table is **built-in** to SQLite:
- ***Verify*** new `table` has been *created* by querying `sqlite_master` 
- It will contain `movie` table definition entry
- Assign to `res`, and ***call*** `res.fetchone()`
    - ***Return*** `('movie',)` or `None` (if non-existant)




In [29]:
res = cur.execute("SELECT name FROM sqlite_master")
print(res.fetchone())
res = cur.execute("SELECT name FROM sqlite_master WHERE name='spam'")
print(res.fetchone() is None)

('movie',)
True


#### 3.3. Inserting Data 
Pattern: 
- `INSERT INTO tbl_nm (col_nm_1, ...) VALUES (val_nm_1, ...)`

Add two rows of data via `INSERT` statement by ***calling*** `cur.execute(...)`

In [31]:
cur.execute("""
    INSERT INTO movie VALUES
        ('Monty Python and the Holy Grail', 1975, 8.2),
        ('And Now for Something Completely Different', 1971, 7.5)
""")

#### 3.4. Commiting Transaction
The INSERT statement ***implicitly*** opens a `transaction`: 
- which ***needs*** to be ***committed*** 
- before changes are saved in the database (see `Transaction` control):

***Call*** `con.commit()` to ***commit*** the `transaction`.


In [32]:
con.commit()

In [33]:
con

#### 3.5. Verifying Commitment
To ***verify*** the inserted data:
- Use `cur.execute(...)` to 
- ***execute*** a `SELECT` query
- ***assign*** result to `res`, 
- ***call*** `res.fetchall()` to return all resulting rows

In [38]:
cur.execute(".mode column")

OperationalError: near ".": syntax error

In [35]:
res = cur.execute("SELECT * FROM movie")
res.fetchall()

[('Monty Python and the Holy Grail', 1975, 8.2),
 ('And Now for Something Completely Different', 1971, 7.5),
 ('Monty Python and the Holy Grail', 1975, 8.2),
 ('And Now for Something Completely Different', 1971, 7.5)]

In [ ]:
import sqlite3

con = sqlite3.connect("tony_OG.db")
cur = con.cursor()

print(f"Connection Established: \t{con} | Connection level (None is auto-commit): {con.isolation_level}")
print(f"Cursor Created:     \t\t{cur}")

### 1. VALIDATE TABLE EXISTENCE
res = cur.execute("SELECT NAME FROM sqlite_master").fetchone()
print(f"1_validate_tbl_existence[sqlite_master]: {res}") #('users',)
# res = cur.execute("SELECT NAME FROM sqlite_master where NAME = 'spam ") # None


### 2. GET DATA from user TBL
### 2a. get all - fetchall()
res = cur.execute("""SELECT * from users""").fetchall()
print(f"2a_select*from_users[fetchall()]: {res}")
print()
# print(f"tony id: {res[0][0]}")
# print(f"tony name: {res[0][1]}")
# print(f"tony age: {res[0][2]}")

### 2b. get one row - fetchone()
res = cur.execute("""SELECT * from users""").fetchone()
print()
print(f"2b_select*from_users[fetchone()]: {res}") 
# print(f"one-row expected: {res}")

### 2c. get some rows - fetchmany(n)
res = cur.execute("""SELECT * from users""").fetchmany(2)
print()
print(f"2c_select*from_users[fetchmany(2)]: {res}") 

#################################################################################

### 3. INSERT INTO user TBL
# 3a INSERT 1 OR FEW: execute without commit

# The INSERT statement implicitly opens a transaction, 
# which needs to be committed before changes are saved in the database 
# Call con.commit() on the connection object to commit the transaction:
cur.execute("INSERT INTO users (name, age) VALUES ('iniesta', 50),('xavi', 80)")


print()
res = cur.execute("SELECT * FROM users").fetchall()
print(f"3a_insert_wout_con_commit: {res}") 


# 3a INSERT MANY with LIST:
input_data_list = [
    ('pedri',123,"default@barca.com"),
    ('gavi',231,"default@barca.com"),
    ('baldi',444,"default@barca.com")
]
cur.executemany("INSERT INTO users (name,age,email) VALUES (?, ?, ?)", input_data_list)

print()
res = cur.execute("SELECT * FROM users").fetchall()
print(f"3b_insert_many_wout_con_commit: {res}") 
con.commit()
con.close()
